### Set Path (Won't be needed once `setup.py` is finished)

In [1]:
import sys
sys.path.append(sys.path[0][:-8])

In [2]:
import torch
from tqdm import tqdm
import numpy as np

from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')                                                                                                                                                       

from rdkit.Chem import Draw
from matplotlib import pyplot as plt

from sklearn.metrics import roc_auc_score as ras
from sklearn.metrics import mean_squared_error

### Auglichem imports

In [3]:
from auglichem.molecule import Compose, RandomAtomMask, RandomBondDelete
from auglichem.molecule.data import MoleculeDatasetWrapper
from auglichem.molecule.models import GCN, AttentiveFP, GINE, DeepGCN

### Set up dataset

In [4]:
help(MoleculeDatasetWrapper)

Help on class MoleculeDatasetWrapper in module auglichem.molecule.data._molecule_dataset:

class MoleculeDatasetWrapper(MoleculeDataset)
 |  MoleculeDatasetWrapper(*args, **kwds)
 |  
 |  Dataset base class for creating graph datasets.
 |  See `here <https://pytorch-geometric.readthedocs.io/en/latest/notes/
 |  create_dataset.html>`__ for the accompanying tutorial.
 |  
 |  Args:
 |      root (string, optional): Root directory where the dataset should be
 |          saved. (optional: :obj:`None`)
 |      transform (callable, optional): A function/transform that takes in an
 |          :obj:`torch_geometric.data.Data` object and returns a transformed
 |          version. The data object will be transformed before every access.
 |          (default: :obj:`None`)
 |      pre_transform (callable, optional): A function/transform that takes in
 |          an :obj:`torch_geometric.data.Data` object and returns a
 |          transformed version. The data object will be transformed before
 |   

In [5]:
help(MoleculeDatasetWrapper.__init__)

Help on function __init__ in module auglichem.molecule.data._molecule_dataset:

__init__(self, dataset, transform=None, split='scaffold', batch_size=64, num_workers=0, valid_size=0.1, test_size=0.1, aug_time=0, data_path=None, target=None, seed=None)
    Input:
    ---
    dataset (str): One of the datasets available from MoleculeNet
                   (http://moleculenet.ai/datasets-1)
    transform (Compose, OneOf, RandomAtomMask, RandomBondDelete object): transormations
                   to apply to the data at call time.
    split (str, optional default=scaffold): random or scaffold. The splitting strategy
                                            used for train/test/validation set creation.
    batch_size (int, optional default=64): Batch size used in training
    num_workers (int, optional default=0): Number of workers used in loading data
    valid_size (float in [0,1], optional default=0.1): 
    test_size (float in [0,1],  optional default=0.1): 
    aug_time (int, optional

In [6]:
help(RandomAtomMask)

Help on class RandomAtomMask in module auglichem.molecule._transforms:

class RandomAtomMask(BaseTransform)
 |  RandomAtomMask(p: float = 1.0)
 |  
 |  Method resolution order:
 |      RandomAtomMask
 |      BaseTransform
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, p: float = 1.0)
 |      @param p: the probability of the transform being applied; default value is 1.0
 |  
 |  apply_transform(self, mol_graph: torch_geometric.data.data.Data, seed: NoneType) -> torch_geometric.data.data.Data
 |      Transform that randomly mask atoms given a certain ratio
 |      @param mol_graph: PyG Data to be augmented
 |      @param seed: 
 |      @returns: Augmented PyG Data
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from BaseTransform:
 |  
 |  __call__(self, mol_graph: torch_geometric.data.data.Data, seed=None) -> torch_geometric.data.data.Data
 |      @param mol_graph: PyG Data to be augmented
 |      @par

In [7]:
# Create transformation
transform = Compose([
    RandomAtomMask([0.1, 0.3]),
    RandomBondDelete([0.1, 0.3])
])

# Initialize dataset object
dataset = MoleculeDatasetWrapper("QM8", data_path="./data_download", transform=transform, batch_size=5)

# Get train/valid/test splits as loaders
train_loader, valid_loader, test_loader = dataset.get_data_loaders("all")

Using: ./data_download/gdb8/qm8.csv
DATASET: QM8


21783it [00:00, 22844.96it/s]


Generating scaffolds...
Generating scaffold 0/21782
Generating scaffold 1000/21782
Generating scaffold 2000/21782
Generating scaffold 3000/21782
Generating scaffold 4000/21782
Generating scaffold 5000/21782
Generating scaffold 6000/21782
Generating scaffold 7000/21782
Generating scaffold 8000/21782
Generating scaffold 9000/21782
Generating scaffold 10000/21782
Generating scaffold 11000/21782
Generating scaffold 12000/21782
Generating scaffold 13000/21782
Generating scaffold 14000/21782
Generating scaffold 15000/21782
Generating scaffold 16000/21782
Generating scaffold 17000/21782
Generating scaffold 18000/21782
Generating scaffold 19000/21782
Generating scaffold 20000/21782
Generating scaffold 21000/21782
About to sort in scaffold sets


### Initialize model with task from data

In [8]:
# Get model
num_outputs = len(dataset.labels.keys())
model = AttentiveFP(task=dataset.task, output_dim=num_outputs)

# Uncomment the following line to use GPU
#model.cuda()

### Initialize traning loop

In [9]:
if(dataset.task == 'classification'):
    criterion = torch.nn.CrossEntropyLoss()
elif(dataset.task == 'regression'):
    criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

### Train the model

In [10]:
for epoch in range(2):
    for bn, data in tqdm(enumerate(train_loader)):
        optimizer.zero_grad()
        loss = 0.
        
        # Get prediction for all data
        _, pred = model(data)
        
        # To use GPU, data must be cast to cuda
        #_, pred = model(data.cuda())

        for idx, t in enumerate(train_loader.dataset.target):
            # Get indices where target has a value
            good_idx = np.where(data.y[:,idx]!=-999999999)
            
            # When the data is placed on GPU, target must come back to CPU
            #good_idx = np.where(data.y.cpu()[:,idx]!=-999999999)

            # Prediction is handled differently for classification and regression
            if(train_loader.dataset.task == 'classification'):
                current_preds = pred[:,2*(idx):2*(idx+1)][good_idx]
                current_labels = data.y[:,idx][good_idx]
            elif(train_loader.dataset.task == 'regression'):
                current_preds = pred[:,idx][good_idx]
                current_labels = data.y[:,idx][good_idx]
            
            loss += criterion(current_preds, current_labels)
        
        loss.backward()
        optimizer.step()

3485it [04:20, 13.38it/s]
3485it [07:55,  7.33it/s]


### Test the model

In [11]:
def evaluate(model, test_loader, validation=False):
    set_str = "VALIDATION" if validation else "TEST"
    with torch.no_grad():
        
        # All targets we're evaluating
        target_list = test_loader.dataset.target
        
        # Dictionaries to keep track of predictions and labels for all targets
        all_preds = {target: [] for target in target_list}
        all_labels = {target: [] for target in target_list}
        
        model.eval()
        for data in test_loader:
            # Get prediction for all data
            _, pred = model(data)

            # To use GPU, data must be cast to cuda
            #_, pred = model(data.cuda())
            
            for idx, target in enumerate(target_list):
                # Get indices where target has a value
                good_idx = np.where(data.y[:,idx]!=-999999999)
                
                # When the data is placed on GPU, target must come back to CPU
                #good_idx = np.where(data.y.cpu()[:,idx]!=-999999999)
                
                # Prediction is handled differently for classification and regression
                if(train_loader.dataset.task == 'classification'):
                    current_preds = pred[:,2*(idx):2*(idx+1)][good_idx][:,1]
                    current_labels = data.y[:,idx][good_idx]
                elif(train_loader.dataset.task == 'regression'):
                    current_preds = pred[:,idx][good_idx]
                    current_labels = data.y[:,idx][good_idx]
                
                # Save predictions and targets
                all_preds[target].extend(list(current_preds.detach().cpu().numpy()))
                all_labels[target].extend(list(current_labels.detach().cpu().numpy()))
            
        scores = {target: None for target in target_list}
        for target in target_list:
            if(test_loader.dataset.task == 'classification'):
                scores[target] = ras(all_labels[target], all_preds[target])
                print("{0} {1} ROC: {2:.5f}".format(target, set_str, scores[target]))
            elif(test_loader.dataset.task == 'regression'):
                scores[target] = mean_squared_error(all_labels[target], all_preds[target],
                                                    squared=False)
                print("{0} {1} RMSE: {2:.5f}".format(target, set_str, scores[target]))

In [12]:
evaluate(model, valid_loader, validation=True)
evaluate(model, test_loader)

E1-CC2 VALIDATION RMSE: 0.02281
E2-CC2 VALIDATION RMSE: 0.02368
f1-CC2 VALIDATION RMSE: 0.04655
f2-CC2 VALIDATION RMSE: 0.07710
E1-PBE0 VALIDATION RMSE: 0.02367
E2-PBE0 VALIDATION RMSE: 0.02238
f1-PBE0 VALIDATION RMSE: 0.03572
f2-PBE0 VALIDATION RMSE: 0.05695
E1-CAM VALIDATION RMSE: 0.02191
E2-CAM VALIDATION RMSE: 0.02015
f1-CAM VALIDATION RMSE: 0.05059
f2-CAM VALIDATION RMSE: 0.06832
E1-CC2 TEST RMSE: 0.02116
E2-CC2 TEST RMSE: 0.02255
f1-CC2 TEST RMSE: 0.05046
f2-CC2 TEST RMSE: 0.06860
E1-PBE0 TEST RMSE: 0.02184
E2-PBE0 TEST RMSE: 0.02129
f1-PBE0 TEST RMSE: 0.04955
f2-PBE0 TEST RMSE: 0.05762
E1-CAM TEST RMSE: 0.02058
E2-CAM TEST RMSE: 0.01961
f1-CAM TEST RMSE: 0.05733
f2-CAM TEST RMSE: 0.06859


### Model saving/loading example

In [13]:
# Save model
torch.save(model.state_dict(), "./saved_models/example_gcn")

In [14]:
# Instantiate new model and evaluate
model = AttentiveFP(task=dataset.task, output_dim=num_outputs)
evaluate(model, test_loader)

E1-CC2 TEST RMSE: 0.25897
E2-CC2 TEST RMSE: 0.20809
f1-CC2 TEST RMSE: 0.09380
f2-CC2 TEST RMSE: 0.06977
E1-PBE0 TEST RMSE: 0.28468
E2-PBE0 TEST RMSE: 0.26335
f1-PBE0 TEST RMSE: 0.09955
f2-PBE0 TEST RMSE: 0.06399
E1-CAM TEST RMSE: 0.16943
E2-CAM TEST RMSE: 0.23965
f1-CAM TEST RMSE: 0.07573
f2-CAM TEST RMSE: 0.11401


In [15]:
# Load saved model and evaluate
model.load_state_dict(torch.load("./saved_models/example_gcn"))
evaluate(model, test_loader)

E1-CC2 TEST RMSE: 0.02116
E2-CC2 TEST RMSE: 0.02255
f1-CC2 TEST RMSE: 0.05046
f2-CC2 TEST RMSE: 0.06860
E1-PBE0 TEST RMSE: 0.02184
E2-PBE0 TEST RMSE: 0.02129
f1-PBE0 TEST RMSE: 0.04955
f2-PBE0 TEST RMSE: 0.05762
E1-CAM TEST RMSE: 0.02058
E2-CAM TEST RMSE: 0.01961
f1-CAM TEST RMSE: 0.05733
f2-CAM TEST RMSE: 0.06859
